In [21]:
import sys
import os
sys.path.append("../code/")
import cv2
from skimage.metrics import structural_similarity as ssim
import numpy as np
from matplotlib import pyplot as plt
from tabulate import tabulate
import os
import pandas as pd
import json
import dataframe_image as dfi
from forwardFv1 import wm_au_emb
from detectFv1 import *
from recoveryFv1 import *
from sklearn.metrics import precision_score,recall_score

In [22]:
root_dir = "D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images"
list_paths = []
for subdir, dirs, files in os.walk(root_dir):
    for file in files:
        list_paths.append(os.path.join(subdir, file))

In [23]:
print(list_paths)

['D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\airplane.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\baboon.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\house.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\lena.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\peppers.png', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\sailboat.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\splash.tiff', 'D:\\Dat_Hoctap\\2023\\Tamper_Recover\\A_Data\\Data8Images\\tiffany.bmp']


In [24]:
list_img = []
for i in range(len(list_paths)):
    img = cv2.imread(list_paths[i])
    list_img.append(img)

In [25]:
key = (13,137,811)
list_w_img=[]
for i in range(len(list_paths)):
    img = cv2.imread(list_paths[i])
    w_img = wm_au_emb(img,key=key)
    list_w_img.append(w_img)

In [26]:
w_PSNR=[]
w_SSIM=[]
for i in range(len(list_img)):
    w_PSNR.append(cv2.PSNR(list_img[i],list_w_img[i]))
    w_SSIM.append(ssim(cv2.cvtColor(list_img[i],cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_w_img[i],cv2.COLOR_BGR2GRAY)))

In [27]:
print(np.mean(w_PSNR))
print(w_SSIM)

46.25427068599326
[0.9926808084152747, 0.9980711604196131, 0.9946383141094459, 0.9937209599890108, 0.9939592251156862, 0.9954525047825709, 0.9917258726174685, 0.9927672299074065]


In [28]:
# df_all_PSNR.to_csv("results/PSNR_Watermarked_Images.csv")
# df_all_ssim.to_csv("results/SSIM_Watermarked_Images.csv")

In [29]:
N = 512
np.random.seed(1)
t_all_image = np.random.randint(
    low=0, 
    high=256,
    size=(N, N, 3),
    dtype=np.uint8
)

In [30]:
list_tr = [10,20,30,40,50,60,70,80]
list_d = []
for i in range(len(list_tr)):
    list_d.append(int(np.sqrt(512*512*list_tr[i]/100)/2+ 0.5))
print(list_d)

[81, 114, 140, 162, 181, 198, 214, 229]


In [31]:
list_PSNR_Recovered_all = []
list_SSIM_Recovered_all = []
list_PSNR_Recovered_all_w_original = []
list_SSIM_Recovered_all_w_original = []
for itr in range(len(list_d)):
    list_PSNR = []
    list_SSIM = []
    list_PSNR_original = []
    list_SSIM_original = []
    for index_img in range(len(list_w_img)):
        t_img = list_w_img[index_img].copy()
        d = list_d[itr]
        t_img[256-d:256+d,256-d:256+d,:] =  t_all_image[256-d:256+d,256-d:256+d,:]
        lv1_matrix = level_one_detection(t_img)
        lv2_matrix = lv1_matrix.copy()
        lv3_matrix = lv1_matrix.copy()
        for channel in range(3):
            lv2_matrix[:,:,channel] = level_two_detection(lv1_matrix[:,:,channel])
            lv3_matrix[:,:,channel] = level_three_detection(lv2_matrix[:,:,channel])
        r1_img,lv4_BGR= stage1_recovery(t_img=t_img,key=(13,137,811))
        r2_img,lv5_BGR = stage2_recovery(r1_img=r1_img,lv4_matrix=lv4_BGR)
        mask1 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv5_BGR[int(i/1),int(j/1),0]:
                    mask1[i,j] = 255
        mask2 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv5_BGR[int(i/1),int(j/1),1]:
                    mask2[i,j] = 255
        mask3 = np.zeros((512,512),dtype = np.uint8)
        for i in range(512):
            for j in range(512):
                if not lv5_BGR[int(i/1),int(j/1),2]:
                    mask3[i,j] = 255
        r3_img = r2_img.copy()
        r3_img[:,:,0] = cv2.inpaint(r2_img[:,:,0],mask1,0,cv2.INPAINT_TELEA)
        r3_img[:,:,1] = cv2.inpaint(r2_img[:,:,1],mask2,0,cv2.INPAINT_TELEA)
        r3_img[:,:,2] = cv2.inpaint(r2_img[:,:,2],mask3,0,cv2.INPAINT_TELEA)
        list_PSNR.append(cv2.PSNR(r3_img,list_w_img[index_img]))
        list_SSIM.append(ssim(cv2.cvtColor(r3_img,cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_w_img[index_img],cv2.COLOR_BGR2GRAY)))
        list_PSNR_original.append(cv2.PSNR(r3_img,list_img[index_img]))
        list_SSIM_original.append(ssim(cv2.cvtColor(r3_img,cv2.COLOR_BGR2GRAY),cv2.cvtColor(list_img[index_img],cv2.COLOR_BGR2GRAY)))
    list_PSNR_Recovered_all.append(list_PSNR)
    list_SSIM_Recovered_all.append(list_SSIM)
    list_PSNR_Recovered_all_w_original.append(list_PSNR_original)
    list_SSIM_Recovered_all_w_original.append(list_SSIM_original)

In [ ]:
list_image_name =  ['Airplane', 'Baboon', 'House', 'Lena','Peppers','Sailboat','Splash','Tiffany','Average']

In [ ]:
list_PSNR_Recovered_all[0]

[]

In [ ]:
np.mean(list_PSNR_Recovered_all_w_original[0])

nan

In [ ]:
print(np.mean(list_PSNR_Recovered_all[2]))

nan


In [ ]:
dict_PSNR = {}
dict_SSIM = {}

In [ ]:
for i in range(8):
    dict_PSNR[list_image_name[i]] = [list_PSNR_Recovered_all[j][i] for j in range(8)]
dict_PSNR[list_image_name[8]] = [np.mean(list_PSNR_Recovered_all[j]) for j in range(8)]

IndexError: list index out of range

In [ ]:
for i in range(8):
    dict_SSIM[list_image_name[i]] = [list_SSIM_Recovered_all[j][i] for j in range(8)]
dict_SSIM[list_image_name[8]] = [np.mean(list_SSIM_Recovered_all[j]) for j in range(8)]

In [ ]:
print(dict_PSNR['Average'])

[39.68521155992695, 37.29263947510198, 33.617306062972276, 29.608528969644226, 26.899189923999934, 25.190987732237566, 23.718890459348714, 22.26769070627691]


In [ ]:
df_PSNR_all = pd.DataFrame.from_dict(dict_PSNR)

In [ ]:
df_SSIM_all = pd.DataFrame.from_dict(dict_SSIM)

In [ ]:
df_SSIM_all.to_csv("SSIM_all_recovered1.csv")

In [ ]:
df_PSNR_all.to_csv("PSNR_all_recovered1.csv")

In [ ]:
def highlight_max(s):
    # Get 5 largest values of the column
    is_large1 = s.nlargest(1).values
    is_large2 = s.nlargest(2).values
    # Apply style is the current value is among the 5 biggest values
    return ['color: DarkRed;font-weight:900' if v in is_large1 else 'font-weight:900' if v in is_large2 else '' for v in s]

In [ ]:
# hl_df_ssim = df_all_ssim.style.apply(highlight_max,subset=df_all_ssim.columns[1:],axis = 1).apply(lambda x: ['background: lightgreen' if x.name == 8 else '' for i in x], 
#                axis=1)

In [ ]:
hl_df_PSNR_all = df_PSNR_all.style.apply(highlight_max,subset=df_PSNR_all.columns[:],axis=1)
hl_df_PSNR_all.format(precision=2)
hl_df_PSNR_all.hide_index()

C:\Users\61092\AppData\Local\Temp\ipykernel_6064\3007742081.py:3: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  hl_df_PSNR_all.hide_index()


Airplane,Baboon,House,Lena,Peppers,Sailboat,Splash,Tiffany,Average
38.60,38.48,37.74,39.68,39.18,37.82,44.61,41.37,39.69
36.69,35.03,35.52,36.87,37.35,35.25,42.33,39.29,37.29
33.53,29.56,32.38,33.20,34.85,31.44,37.09,36.90,33.62
29.41,25.37,27.84,29.73,30.84,27.23,32.39,34.07,29.61
26.60,22.76,25.07,27.92,28.22,24.29,29.22,31.11,26.90
24.95,21.13,23.71,26.09,26.16,22.87,27.62,29.00,25.19
23.52,19.79,22.18,24.81,24.10,21.51,26.38,27.47,23.72
22.18,18.50,20.85,23.27,22.30,19.95,24.98,26.11,22.27


In [ ]:
hl_df_SSIM_all = df_SSIM_all.style.apply(highlight_max,subset=df_SSIM_all.columns[:],axis=1)
hl_df_SSIM_all.format(precision=4)
hl_df_SSIM_all.hide_index()

C:\Users\61092\AppData\Local\Temp\ipykernel_6064\3815801646.py:3: FutureWarning: this method is deprecated in favour of `Styler.hide(axis="index")`
  hl_df_SSIM_all.hide_index()


Airplane,Baboon,House,Lena,Peppers,Sailboat,Splash,Tiffany,Average
0.9910,0.9835,0.9888,0.9903,0.9862,0.9859,0.9928,0.9887,0.9884
0.9843,0.9669,0.9772,0.9793,0.9733,0.9748,0.9857,0.9790,0.9775
0.9720,0.9334,0.9600,0.9612,0.9552,0.9552,0.9746,0.9653,0.9596
0.9476,0.8702,0.9251,0.9312,0.9264,0.9203,0.9557,0.9423,0.9273
0.9123,0.7904,0.8795,0.8914,0.8883,0.8698,0.9288,0.9047,0.8831
0.8798,0.7088,0.8376,0.8497,0.8504,0.8199,0.9010,0.8643,0.8389
0.8338,0.6089,0.7766,0.8059,0.7953,0.7566,0.8647,0.8151,0.7821
0.7743,0.4843,0.7012,0.7376,0.7290,0.6671,0.8143,0.7558,0.7079


In [ ]:
# hl_df_ssim.format(precision=4)
# hl_df_ssim.hide_index()

In [ ]:
dfi.export(
    hl_df_PSNR_all,
    "results1/SSIM_recovered_Images.png",
    fontsize=12,
)

In [ ]:
dfi.export(
    hl_df_SSIM_all,
    "results1/SSIM_recovered_Images.png",
    fontsize=12,
)

c:\ProgramData\Anaconda3\envs\ENV_Common\Lib\site-packages\dataframe_image\_pandas_accessor.py:69: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html = '<div>' + obj.render() + '</div>'


SyntaxError: not a PNG file (<string>)